![Alt text](http://www.ucm.es/logo/ucm.png "a title")

<div align="center"> 
<font size=5> Máster en Nuevas Tecnologías Electrónicas y Fotónicas </font>
</div>
    
<div align="center"> 
<font size=4> Óptica Digital, curso 2022-2023 </font>
</div>

    
<div align="center"> 
<font size=4>   </font>
</div>


<div align="center"> 
<font size=5> Ejercicio 4 - Elementos Ópticos Difractivos </font>
</div>

- **Fecha**: 
        
- **Alumno**: 


# Introducción

En estos ejercicios nos vamos a centrar en:

- funcionamiento del áxicon. 
- Array de multilentes (opcional).


Proponga un valor para los parámetros que no vengan en el enunciado. En todos los casos, los resultados son gráficas de intensidad y fase luminosa.

<div class="alert alert-block alert-success">
    
<b>Nota:</b>
Para determinar la posición central lo puede hacer mediante:
    
- i_centro=int(numdatos/2)
- j_centro=int(numdatos/2)    
    
    
Para calcular la intensidad, se puede utilizar la propiedad intensity() del marco XY.    
    
</div>

In [ ]:
from matplotlib import rcParams

rcParams["figure.dpi"] = 250

from diffractio.utils_drawing import draw_several_fields

from dataclasses import dataclass, replace, field
from typing import NamedTuple, Tuple, Callable, Protocol
from diffractio.scalar_sources_XY import Scalar_source_XY
from diffractio.scalar_masks_XY import Scalar_mask_XY

from diffractio import degrees, mm, um, nm
from diffractio.scalar_fields_XY import Scalar_field_XY

from matplotlib import pyplot as plt
import numpy as np

from numpy.typing import NDArray


def draw_intensity_phase_XY(field: Scalar_mask_XY, title: str):
    # Bug: Draw_several_fields only works for XY
    draw_several_fields(
        fields=(field, field), kinds=["intensity", "phase"], title=title, logarithm=1e2
    )
    axes = plt.gca()
    axes.figure._suptitle.set_fontsize(12)

    plt.show()


@dataclass(frozen=True)
class SimulationConfig:
    radius: float = 0.5 * mm
    resolution: int = 512
    wavelength: float = 600 * nm


class XYFactory(Protocol):
    def __call__(
        self,
        x: NDArray[np.floating],
        y: NDArray[np.floating],
        *,
        simulation: SimulationConfig,
    ) -> Scalar_field_XY:
        ...


class XYMeshFactory(Protocol):
    def __call__(
        self, *, simulation: SimulationConfig
    ) -> Tuple[NDArray[np.floating], NDArray[np.floating]]:
        ...


class PropagationFactory(Protocol):
    def __call__(
        self, field: Scalar_field_XY, *, simulation: SimulationConfig
    ) -> Scalar_field_XY:
        ...


def maskFactory(x, y, *, simulation=SimulationConfig()):
    mask = Scalar_mask_XY(x, y)
    mask.one_level(1)
    return mask


def waveFactory(x, y, *, simulation=SimulationConfig()):
    u1 = Scalar_source_XY(x=x, y=y, wavelength=simulation.wavelength)
    u1.plane_wave(A=1, theta=0, phi=0)
    return u1


def defaultMeshFactory(*, simulation: SimulationConfig):
    x = np.linspace(
        -simulation.radius,
        simulation.radius,
        simulation.resolution,
    )
    y = np.linspace(
        -simulation.radius,
        simulation.radius,
        simulation.resolution,
    )
    return x, y


@dataclass(frozen=True)
class SimulationParams:
    observation_distance: float = None  # defaults to the focal length
    focal: float = 10 * mm
    lens_diameter: float = 0.5 * mm


RunResult = NamedTuple(
    "RunResult", incoming=Scalar_field_XY, observation=Scalar_field_XY
)


@dataclass(frozen=True)
class SimulationPropagator:
    propagation: PropagationFactory
    mask: XYFactory = maskFactory
    wave: XYFactory = waveFactory
    mesh: XYMeshFactory = defaultMeshFactory

    def run(self, config: SimulationConfig = SimulationConfig()):
        x, y = self.mesh(simulation=config)
        wave = self.wave(x, y, simulation=config)
        mask = self.mask(x, y, simulation=config)
        incoming = wave * mask
        observation = self.propagation(incoming)
        return RunResult(incoming, observation)

    @staticmethod
    def plot(field: Scalar_field_XY, *, title=None):
        draw_intensity_phase_XY(
            field,
            title=title,
        )
        plt.show()


# 1. Axicon


1. **Axicón**: Vamos a definir un axicon binario en el marco XY y de fase. Podemos utilizar un tamaño de 0.5 mm, 256 puntos, un ángulo (del axicon refractivo de 10º, y un índice de refracción equivalente de n=1.5. Dicho axicón se ilumina con una onda armónica plana en incidencia normal

  - Determine la distribución de intensidad generada por el axicon a una distancia z=2.5 mm. Dibújela.
 
  - Queremos saber la intensidad del pico central (x=0, y=0), en un intervalo entre z = 0.125 mm y 3.5 mm. Haga un bucle for entre estos valores y almacene la intensidad central en un array
 
  - Dibuje la intensidad máxima en función de la distancia



In [ ]:
@dataclass
class SimulationParams:
    radius:float = 0.4 * mm
    angle:float = 15 * degrees
    n:float = 1.6
    z:float = 2 * mm


def run(params=SimulationParams(), config=SimulationConfig()):
    def mask(x, y, *, simulation: SimulationConfig):
        r0 = (0, 0)
        axicon = Scalar_mask_XY(x, y, wavelength=simulation.wavelength)

        axicon.axicon(
            r0, refraction_index=params.n, radius=params.radius, angle=params.angle
        )
        return axicon

    def propagation(field: Scalar_field_XY, **_):
        observation = field.RS(
            params.z,
            new_field=True,
            verbose=True,
        )
        return observation

    simulation = SimulationPropagator(propagation, mask=mask)
    return simulation.run(config)


incoming, observation = run(config=SimulationConfig(resolution=2048))


In [ ]:

SimulationPropagator.plot(incoming, title="Incoming field")
SimulationPropagator.plot(observation, title="Outgoing field")


Ahora, hagamos el calculo en la linea central $x=y=0$ en un rango de z amplio para ver la amplitud


In [ ]:
incoming, observation = run(
    config=SimulationConfig(resolution=2400), params=SimulationParams(z=0.15)
)


In [ ]:
#  z = 0.125 mm y 3.5 mm.
from itertools import chain, repeat


def field_z_line(zs, resolutions, config=SimulationConfig(), params=SimulationParams()):
    index_x = None
    index_y = None
    # Fixed

    for z, resolution in zip(zs, resolutions):
        # the print of the factor uses a \r, so they replace each other
        # Make resolution an odd integer
        incoming, observation = run(
            config=replace(config, resolution=resolution), params=replace(params, z=z)
        )
        print("\n\n")
        observation: Scalar_field_XY
        # Find center, assume odd resolution
        index_x = len(observation.x) // 2 
        index_y = len(observation.y) // 2 
        yield observation.u[index_x, index_y]


N = 70
zs = np.linspace(0.2 * mm, 4 * mm, N)
resolutions = [*chain(repeat(2500, N//4), repeat(700, N - N//4))]


field_z_axis = np.fromiter(field_z_line(zs, resolutions), dtype=np.csingle)


Realizamos estimaciones con distinta resoluciones para obtener un factor bueno al principio.

Sin embargo, al hacer este cambio de densidad, tenemos que corregir de la solucion el cambio de intensidad por cada pixel indica. Para ello, hacemos una correccion teniendo en cuenta que la resolucion indica la densidad lineal de pixeles.  

In [ ]:
from diffractio.scalar_fields_X import Scalar_field_X

energy_density = np.array([r**(-2/3) for r in resolutions])

f = Scalar_field_X(zs)
f.u = field_z_axis * energy_density

f.draw()
plt.xlabel("z ($\mu m$)")
plt.ylabel("Proportional to intensity")
plt.title(label="Wave Intensity of cross section on the propagation axis of ")
plt.show()


**Conclusiones**:

El axion permite una colimacion en el haz principal muy intensa que ocurre durante una distancia larga. Notese que en la simulacion entre 2000 y 2500 se deben a que esta colimado a un area muy pequeña, y es posible que no estemos cogiendo el centro del haz muy bien en esos casos.

# 2. Array de multilentes


2. **Array de multilentes** (opcional): Cree una máscara que sea un array de multilentes de 7x7 elementos y un tamaño de 2 mm. Dicha máscara se ilumina con una onda esférica convergente situada a z=25 mm de la máscara. 

  - Dibuje la máscara generada.
  
  - Dibuje la distribución de intensidad en el plano focal.
  
  Para generar la máscara, utilice un doble bucle for.

In [ ]:
#TBH, framework is ready above

### Propagación

**Conclusiones**: